In [3]:
import os
file_paths = []
folder_path = '../data_base/knowledge_db'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

In [4]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))

In [5]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)

In [12]:
# import zhipuai_embedding
from zhipuai_embedding import ZhipuAIEmbeddings

In [13]:
import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())
print(os.getenv('ZHIPUAI_API_KEY'))

30c75522d7b003b86327078454b70118.88NFc995EqyzvrUT


In [14]:
embedding = ZhipuAIEmbeddings()
# 定义持久化路径
persist_directory = '../data_base/vector_db/chroma'

In [ ]:
# !rm -rf '../data_base/vector_db/chroma'

'rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [19]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[20:], # 为了速度，只选择前 20 个切分的 doc 进行生成；使用千帆时因QPS限制，建议选择前 5 个doc
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [20]:
vectordb.persist()

In [22]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1004


In [23]:
# 在Chroma数据库中进行检索
question = '什么是大语言模型'

sim_docs = vectordb.similarity_search(question, k=3)


In [26]:
for i, sim_doc in enumerate(sim_docs):
    print(f'检索到的第{i}个内容:\n{sim_doc.page_content[:200]}', end='\n --------------------------\n')

检索到的第0个内容:
网络上有许多关于提示词（Prompt， 本教程中将保留该术语）设计的材料，例如《30 prompts everyone has to know》之类的文章，这些文章主要集中在 ChatGPT 的 Web 界面上，许多人在使用它执行特定的、通常是一次性的任务。但我们认为，对于开发人员，大语言模型（LLM） 的更强大功能是能通过 API 接口调用，从而快速构建软件应用程序。实际上，我们了解到 Deep
 --------------------------
检索到的第1个内容:
第六章 文本转换

大语言模型具有强大的文本转换能力，可以实现多语言翻译、拼写纠正、语法调整、格式转换等不同类型的文本转换任务。利用语言模型进行各类转换是它的典型应用之一。

在本章中,我们将介绍如何通过编程调用API接口，使用语言模型实现文本转换功能。通过代码示例，读者可以学习将输入文本转换成所需输出格式的具体方法。

掌握调用大语言模型接口进行文本转换的技能，是开发各种语言类应用的重要一步。文
 --------------------------
检索到的第2个内容:
与基础语言模型不同，指令微调 LLM 通过专门的训练，可以更好地理解并遵循指令。举个例子，当询问“法国的首都是什么？”时，这类模型很可能直接回答“法国的首都是巴黎”。指令微调 LLM 的训练通常基于预训练语言模型，先在大规模文本数据上进行预训练，掌握语言的基本规律。在此基础上进行进一步的训练与微调（finetune），输入是指令，输出是对这些指令的正确回复。有时还会采用RLHF（reinforce
 --------------------------


In [27]:
# 使用最大边际相关性进行检索
mmr_docs = vectordb.max_marginal_relevance_search(question, k=3)

In [28]:
for i, mmr_doc in enumerate(mmr_docs):
    print(f'检索到的第{i}个内容：\n{mmr_doc.page_content[:200]}', end='\n ---------------------------- \n')

检索到的第0个内容：
网络上有许多关于提示词（Prompt， 本教程中将保留该术语）设计的材料，例如《30 prompts everyone has to know》之类的文章，这些文章主要集中在 ChatGPT 的 Web 界面上，许多人在使用它执行特定的、通常是一次性的任务。但我们认为，对于开发人员，大语言模型（LLM） 的更强大功能是能通过 API 接口调用，从而快速构建软件应用程序。实际上，我们了解到 Deep
 ---------------------------- 
检索到的第1个内容：
学生计算的总费用：450x+10万美元
实际计算的总费用：360x+10万美元
学生计算的费用和实际计算的费用是否相同：否
学生的解决方案和实际解决方案是否相同：否
学生的成绩：不正确

三、局限性

开发大模型相关应用时请务必铭记：

虚假知识：模型偶尔会生成一些看似真实实则编造的知识

在开发与应用语言模型时，需要注意它们可能生成虚假信息的风险。尽管模型经过大规模预训练，掌握了丰富知识，但它实
 ---------------------------- 
检索到的第2个内容：
行推导。
对于任意样本, 在不考虑样本本身之前(即先验), 若瞎猜一下它由第i 个高斯混合成分生成的概率
P (zj = i), 那么肯定按先验概率α1, α2, . . . , αk 进行猜测, 即P (zj = i) = αi 。若考虑样本本身带来的信
息(即后验), 此时再猜一下它由第i 个高斯混合成分生成的概率pM (zj = i | xj), 根据贝叶斯公式, 后验概
率pM (zj =
 ---------------------------- 
